# Data Augmentation

## 0. Import Library

In [1]:
import albumentations as A
import numpy as np
import seedir as sd
import os
import glob
from sklearn.model_selection import train_test_split
import shutil

## 1. Synthetic Data Generation

### Video

#### Splitting into Training and Validation

In [ ]:
video_dir = r'D:\RESEARCH ASSISTANT\6. Depth Camera\CODE\Orbbec Gemini 2XL\REMOTE\DEVELOPMENT\notebook\DATA\20250402\video\rgb'
save_dir = os.path.join(os.path.dirname(video_dir), 'split_rgb')

# Define subfolders
train_dir = os.path.join(save_dir, 'train')
val_dir = os.path.join(save_dir, 'val')

# Create directory structure
for target_dir in [train_dir, val_dir]:
    os.makedirs(target_dir, exist_ok=True)

# Loop over each class folder
for class_name in os.listdir(video_dir):
    class_path = os.path.join(video_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    # List all .npy files in the class
    npy_files = glob.glob(os.path.join(class_path, '*.npy'))

    # Split using sklearn
    train_files, val_files = train_test_split(npy_files, test_size=0.33, random_state=42)

    # Destination subfolders
    train_class_dir = os.path.join(train_dir, class_name)
    val_class_dir = os.path.join(val_dir, class_name)
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(val_class_dir, exist_ok=True)

    # Copy files to train
    for f in train_files:
        shutil.copy2(f, os.path.join(train_class_dir, os.path.basename(f)))

    # Copy files to val
    for f in val_files:
        shutil.copy2(f, os.path.join(val_class_dir, os.path.basename(f)))

print(f"Dataset split complete. Saved to: {save_dir}")

#### Data Generation

1. Use `albumentations` library to generate data of `train` 📁
2. The data shape is `(T, C, H, W)`

In [5]:
video_dir     = r'D:\RESEARCH ASSISTANT\6. Depth Camera\CODE\Orbbec Gemini 2XL\REMOTE\DEVELOPMENT\notebook\DATA\20250402\video' 
train_dir     = os.path.join(video_dir, "split_rgb", "train")
aug_dir       = os.path.join(os.path.dirname(train_dir), "train_aug")
n_applications = 1   # how many aug variants per original

transform = A.ReplayCompose([
    A.ElasticTransform(alpha=0.5, p=0.5),
    A.ShiftScaleRotate(scale_limit=0.05, rotate_limit=10, p=0.5),
    A.RGBShift(r_shift_limit=50, g_shift_limit=50, b_shift_limit=50, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.CLAHE(p=0.5),
    A.PixelDropout(drop_value=0, dropout_prob=0.01, p=0.5),
    A.PixelDropout(drop_value=255, dropout_prob=0.01, p=0.5),
    A.Blur(blur_limit=(2, 4), p=0.5)
])

def augment_numpy_video(arr: np.ndarray):
    """
    arr: numpy video of shape (T, C, H, W), dtype uint8 or float [0,255]
    returns a new numpy array of same shape
    """
    T, C, H, W = arr.shape
    out_frames = []
    replay = None

    for t in range(T):
        # convert (C,H,W) -> (H,W,C) and uint8
        frame = arr[t].transpose(1, 2, 0)
        frame = frame.astype("uint8")

        if t == 0:
            data   = transform(image=frame)
            new    = data["image"]
            replay = data["replay"]
        else:
            new = A.ReplayCompose.replay(replay, image=frame)["image"]

        out_frames.append(new)

    # stack back into (T,C,H,W)
    out_arr = np.stack([f.transpose(2, 0, 1) for f in out_frames], axis=0)
    return out_arr

# 3) Walk each class folder
for cls in os.listdir(train_dir):
    src_cls_dir = os.path.join(train_dir, cls)
    dst_cls_dir = os.path.join(aug_dir, cls)
    os.makedirs(dst_cls_dir, exist_ok=True)

    for fname in os.listdir(src_cls_dir):
        if not fname.endswith(".npy"):
            continue
        src_path = os.path.join(src_cls_dir, fname)
        base, _  = os.path.splitext(fname)

        # load the original video array
        arr = np.load(src_path)

        # save the original in the augmented folder too (optional)
        np.save(os.path.join(dst_cls_dir, f"{base}_orig.npy"), arr)

        # generate N augmented versions
        for i in range(1, n_applications+1):
            aug_arr = augment_numpy_video(arr)
            out_fname = f"{base}_aug{i}.npy"
            out_path  = os.path.join(dst_cls_dir, out_fname)
            np.save(out_path, aug_arr)

    print(f"Augmented class '{cls}': {len(os.listdir(dst_cls_dir))} files")

print("Done creating synthetic training videos.")

Augmented class 'Hand_Close': 4 files
Augmented class 'Hand_Open': 4 files
Augmented class 'Hook_Hand': 4 files
Augmented class 'Intrinsic_Plan': 4 files
Augmented class 'Straight_Fist': 4 files
Done creating synthetic training videos.


#### Visualize Augmentation

In [12]:
import rerun.blueprint as rrb
import rerun as rr

def rerun_visualization_from_npy(npy_path: str):
    # Load the .npy video: shape (T, C, H, W)
    video_array = np.load(npy_path)
    print(f"Loaded {npy_path}, shape: {video_array.shape}")

    stream = rr.new_recording("rerun_augmented_video", spawn=True)

    # Configure layout
    blueprint = rrb.Blueprint(
        rrb.Grid(
            rrb.Vertical(
                rrb.Spatial2DView(origin="/color_image"),
            ),
        ),
        collapse_panels=True,
    )

    # Log each frame
    for idx, frame in enumerate(video_array):
        # Convert (C, H, W) to (H, W, C) for display
        image = np.transpose(frame, (1, 2, 0)).astype(np.uint8)
        stream.set_time_sequence("frame", idx)
        stream.log("color_image", rr.Image(image))

    stream.send_blueprint(blueprint)

classes = ['Hand_Close', 'Hand_Open', 'Hook_Hand', 'Intrinsic_Plan', 'Straight_Fist']
target_dir = os.path.join(aug_dir, classes[0])

# Files with _aug{i} suffix
augmented_files = glob.glob(os.path.join(target_dir, '*_aug*.npy'))

# Visualize first one
rerun_visualization_from_npy(augmented_files[0])

Loaded D:\RESEARCH ASSISTANT\6. Depth Camera\CODE\Orbbec Gemini 2XL\REMOTE\DEVELOPMENT\notebook\DATA\20250402\video\split_rgb\train_aug\Hand_Close\recording_1_Hand_Close_aug1.npy, shape: (16, 3, 300, 300)
